In [1]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler,RobustScaler

In [2]:
fe = "001"
if not os.path.exists(f"../out/fe/fe{fe}"):
    os.makedirs(f"../out/fe/fe{fe}")
    os.makedirs(f"../out/fe/fe{fe}/save")

In [3]:
df = pd.read_csv('../data/train/train_info.csv')

In [4]:
feature_arr = []
target_arr = []
mask_arr = []
target_mask_arr = []
id_list = []
player_list = []

In [5]:
seq_len = 500
shift = 250
offset = 100

In [6]:
print(60 / 44)
print(60 / 64)
print((60 / 44 + 60 / 64) / 2 * 85)


1.3636363636363635
0.9375
97.79829545454544


In [7]:
for index, row in df.iterrows():
    unique_id = row['unique_id']
    player_id = row['player_id']
    mode = row['mode'] - 1
    gender = row['gender'] - 1
    hand = row['hold racket handed'] - 1
    year = row['play years']
    level = row['level'] - 2
    
    filepath = f'../data/train/train_data/{unique_id}.txt'
    imu_data = np.loadtxt(filepath)
    
    batch = len(imu_data) // shift
    feature_arr_ = np.zeros((batch, seq_len, 24))
    target_arr_ = np.zeros((batch, seq_len, 11), dtype=int)
    mask_arr_ = np.zeros((batch, seq_len), dtype=int)
    target_mask_arr_ = np.zeros((batch, seq_len), dtype=int)
    
    # print(imu_data[:10])
    diff_prev = np.diff(imu_data, axis=0, prepend=imu_data[:1])
    # print(diff_prev[:10])
    diff_next = np.diff(imu_data, axis=0, append=imu_data[-1:])
    # print(diff_next[:10])
    cumsum = np.cumsum(imu_data, axis=0)
    # print(cumsum[:10])
    all_feature = np.concatenate((imu_data, diff_prev, diff_next, cumsum), axis=1)
    sc = RobustScaler()
    all_feature = sc.fit_transform(all_feature)
    gender_one_hot = np.eye(2)[gender]
    hand_one_hot = np.eye(2)[hand]
    year_one_hot = np.eye(3)[year]
    level_one_hot = np.eye(4)[level]
    target = np.concatenate((gender_one_hot, hand_one_hot, year_one_hot, level_one_hot))[None, None, :]
    
    for i in range(batch):
        if i == batch - 1:
            fe = all_feature[i*shift:]
            feature_arr_[i, :len(fe), :] = fe
            target_arr_[i, :len(fe), :] = target
            mask_arr_[i, :len(fe)] = 1
            target_mask_arr_[i, offset:len(fe)] = 1
        else:
            feature_arr_[i, :, :] = all_feature[i*shift:i*shift+seq_len]
            target_arr_[i, :, :] = target
            mask_arr_[i, :] = 1
            target_mask_arr_[i, offset:] = 1
        
    feature_arr.append(feature_arr_)
    target_arr.append(target_arr_)
    mask_arr.append(mask_arr_)
    target_mask_arr.append(target_arr_)
    id_list.extend([unique_id for _ in range(batch)])
    player_list.extend([player_id for _ in range(batch)])

    # break

In [8]:
feature_arr = np.concatenate(feature_arr, axis=0)
target_arr = np.concatenate(target_arr, axis=0)
mask_arr = np.concatenate(mask_arr, axis=0)
target_mask_arr = np.concatenate(target_mask_arr, axis=0)

In [9]:
# id_list = np.array(id_list, dtype=int)
# player_list = np.array(player_list, dtype=int)

In [ ]:
fe = "001"
np.save(f"../out/fe/fe{fe}/feature_arr.npy", feature_arr)
np.save(f"../out/fe/fe{fe}/target_arr.npy", target_arr)
np.save(f"../out/fe/fe{fe}/mask_arr.npy", mask_arr)
np.save(f"../out/fe/fe{fe}/target_mask_arr.npy", target_mask_arr)

In [14]:
id_list = np.array(id_list, dtype=int)
player_list = np.array(player_list, dtype=int)
np.save(f"../out/fe/fe{fe}/id_list.npy", id_list)
np.save(f"../out/fe/fe{fe}/player_list.npy", player_list)